# This simplest Pytorch Example of them all

Train a pytorch model to predict 

`f(x) = 2*x + 1`

Should only require a single layer

     input --> bias + weight * x --> output


In [1]:
import torch
import torch.nn as nn
print(torch.__version__)

2.4.0+cu121


## Create Training data

* Inputs = some numbers.
* Outputs = 2*input + 1


In [5]:
training_inputs  = torch.tensor([[0,1,2,3,4,5,6,7,8,9]]).float().T

target = training_inputs * 2 + 1

print(f"inputs = {training_inputs.T.int()}\noutputs = {target.T.int()}")

inputs = tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], dtype=torch.int32)
outputs = tensor([[ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]], dtype=torch.int32)


## Create the simplest pytorch neural network ever.   1 neuron. 

In [6]:
# simplest model ever a 1x1 single layer network
layer1    = nn.Linear(1,1, bias=True)
model     = nn.Sequential(layer1)


## Train the model.

Should be very fast to train a one-dimensional linear (or not quite linear, because of the +1 .. I guess affine) mapping.

In [7]:
## training

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for i in range(100):
    model = model.train()
    
    outputs = model(training_inputs)
    loss = criterion(outputs, target)     # equivalent to: loss = torch.mean((output-target) ** 2)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    if i % 10 == 0:
        print('Epoch: %d | Loss: %.4f' %(i, loss.detach().item()))

Epoch: 0 | Loss: 62.8532
Epoch: 10 | Loss: 0.0017
Epoch: 20 | Loss: 0.0015
Epoch: 30 | Loss: 0.0013
Epoch: 40 | Loss: 0.0012
Epoch: 50 | Loss: 0.0011
Epoch: 60 | Loss: 0.0009
Epoch: 70 | Loss: 0.0008
Epoch: 80 | Loss: 0.0008
Epoch: 90 | Loss: 0.0007


# Verify that the loss is good

In [8]:
loss

tensor(0.0006, grad_fn=<MseLossBackward0>)

## Inspect the model paramters.
Make sure it really did something equivalent to `times 2 plus 1`

In [10]:
list(model.parameters())

[Parameter containing:
 tensor([[2.0073]], requires_grad=True),
 Parameter containing:
 tensor([0.9544], requires_grad=True)]

## Run inferrence!

See if it can predict `10 * 2 + 1`

In [11]:
## test the model
sample = torch.tensor([10.0], dtype=torch.float)
predicted = model(sample)
print(predicted.detach().item())

21.4364070892334


# inspect the parameters a different way

In [13]:
l0 = [c for c in model.children()][0]

f"After training: weight: {l0.weight.item()} bias {l0.bias.item()}"

'After training: weight: 2.1170175075531006 bias 0.2662309408187866'

## Prepare random test data

In [14]:
#Test on random test data.

test_inputs      = torch.rand(5,1)  # torch.linspace(0,9,steps=10).view(-1, 1)
expected_results = test_inputs * 2 + 1


In [16]:
result = model(test_inputs).detach()
print(result)
print(expected_results)

tensor([[0.5323],
        [1.2729],
        [1.4577],
        [0.3285],
        [1.3650]])
tensor([[1.2513],
        [1.9511],
        [2.1256],
        [1.0588],
        [2.0381]])


In [145]:
#result.allclose(test_outputs,result)

## Yey

Notes:

* These are 3 different steps because if you have large training data sets you might run loss.backward() many times (which accumulates gradients) before running optimizer.step()


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
